<a href="https://colab.research.google.com/github/glgunderson/INFOB2DA-PA2/blob/main/pa2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 0 - Setup Environment

In [132]:
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.cluster import AffinityPropagation


# Upload the dataset
url = "https://raw.githubusercontent.com/glgunderson/INFOB2DA-PA2/main/data/online_shoppers_intention.csv"
df = pd.read_csv(url)

print("Dataset loaded, shape:", df.shape) # Number of rows and columns


Dataset loaded, shape: (12330, 18)


The dataset consists of 12,330 records (sessions), including 18 key features for each record.

# Task 1 - Get Dataset on screen

## 1.1) Explore Dataset

In [120]:
df.head() # Show first five rows

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


#### Summary Statitics:

In [121]:
df.info() # Summary of dataset
df.isna().sum() # Check for missing values
df.describe() # Basic statitics for numeric columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


#### Feature Types:

- **Numeric features**  
  `Administrative`, `Administrative_Duration`, `Informational`, `Informational_Duration`,  
  `ProductRelated`, `ProductRelated_Duration`, `BounceRates`, `ExitRates`, `PageValues`, `SpecialDay`

- **Categorical features**  
  `Month`, `OperatingSystems`, `Browser`, `Region`, `TrafficType`, `VisitorType`

- **Boolean features**  
  `Weekend`, `Revenue` (target)

In [122]:
df.describe(include='bool')    # summary counts for booleans

,Weekend,Revenue
count,12330,12330
unique,2,2
top,False,False
freq,9462,10422


In [123]:
categorical_features = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType']

for col in categorical_features:
    df[col] = df[col].astype('category')

# Summary stats for categorical features
df.describe(include='category')

,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType
count,12330,12330,12330,12330,12330,12330
unique,10,8,13,9,20,3
top,May,2,2,1,2,Returning_Visitor
freq,3364,6601,7961,4780,3913,10551


#### Interpretation of Categorical and Boolean Summaries:

- **Month** - 10 unique months are represented, with **May** being the most common (3,364 sessions).  
- **OperatingSystems** - 8 categories, with **OS = 2** used in more than half of the sessions (6,601).  
- **Browser** - 13 browser categories, but **Browser = 2** dominates heavily (7,961 users).  
- **Region** - 9 regions, with **Region = 1** accounting for 4,780 sessions.  
- **TrafficType** - 20 traffic sources, with **TrafficType = 2** the most frequent (3,913).  
- **VisitorType** - 3 categories, with **Returning_Visitor** making up the vast majority (10,551 out of 12,330).  

- **Weekend (Boolean)** - Most sessions occur on weekdays (9,462 False vs. 2,384 True).  
- **Revenue (Boolean)** - Only a small proportion of sessions end in purchase (2,279 True vs. 10,051 False).  

## 1.2) 'Browser 13' vs. 'Other Browsers'

#### Browsers vs. PageValues

In [124]:
# Create dataframe that only consists of Browser and PageValues
df2 = df.loc[:,["Browser","PageValues"]]

# Ensure correct order of labels:
df2['Browser'] = df['Browser'].astype(str)
Browser_labels = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13"]
df2['Browser'] = pd.Categorical(df2['Browser'], categories = Browser_labels)

# Illustrate Browser vs. PageValues
px.bar(
    data_frame = df2.groupby(['Browser'],observed=False).mean().reset_index(),
    x = 'Browser',
    y = 'PageValues',
    color='Browser',
    labels={'Browser':'Browsers','PageValues':'Average PageValue'},
    title="Average PagesValue per. Browser"
    )

#### BrowserGroup vs. PageValues

In [125]:
df2.loc[df2['Browser'] == "13", 'BrowserGroup'] = 'Browser 13'
df2.loc[df2['Browser'] != "13", 'BrowserGroup'] = 'Other Browsers'

df2 = df2.loc[:,["BrowserGroup","PageValues"]]

px.bar(
    data_frame = df2.groupby(['BrowserGroup'],observed=False).mean().reset_index(),
    x = 'BrowserGroup',
    y = 'PageValues',
    color='BrowserGroup',
    labels={'Browser':'Browsers','PageValues':'Average PageValue'},
    title="Average PagesValue per. Browser Group"
    )


#### Browsers vs.

# Task 2 - Preprocessing

In [131]:
df_Norm = df.copy(True)

# Log normalization
def log_normalization(column: pd.Series) -> pd.Series:
    min = column.min()
    max = column.max()

    c = 0000000000000.1

    normalized = (np.log(column + c) - np.log(min + c)) / (np.log(max + c) - np.log(min + c))

    return normalized

# Convert Page type counts to floats
df_Norm['Administrative'] = df_Norm['Administrative'].astype(float)
df_Norm['Informational'] = df_Norm['Informational'].astype(float)
df_Norm['ProductRelated'] = df_Norm['ProductRelated'].astype(float)

df_Norm = df_Norm.select_dtypes(include=['float'])

for c in df_Norm.columns:
  df_Norm[c] = log_normalization(df_Norm[c])

df_Norm.describe()


,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,0.333843,0.334955,0.117124,0.126915,0.573069,0.607312,0.137934,0.287252,0.135450,0.079586
std,0.328444,0.331471,0.230356,0.264832,0.142232,0.185979,0.253236,0.247471,0.262724,0.242791
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.481065,0.562414,0.000000,0.121546,0.000000,0.000000
50%,0.428034,0.415069,0.000000,0.000000,0.586677,0.650612,0.027899,0.204252,0.000000,0.000000
75%,0.662887,0.655468,0.000000,0.000000,0.670675,0.717468,0.141452,0.369070,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Task 3 - Clustering Algorithms

### 3.1) Affinity Propagation Clustering

### 3.2) DBSCAN Clustering

### 3.3) Birch Clustering

# Task 4 - Evaluation of Clustering Methods

### 4.1) Affinity Propagation Clustering (Evaluation)

### 4.2) DBSCAN Clustering (Evaluation)

### 4.3) Birch Clustering (Evaluation)

# Task 5 - Distance Functions

### 5.1) Euclidean Distance Function

### 5.2) Manhatten Distance Function

### 5.3) Cosine Similarity Function